# Neurosynth

See if we get interesting results in neurosynth

In [2]:
# Import modules and R
import numpy as np, pandas as pd
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

In [5]:
# Import local code
import sys
sys.path.append('../code')
from gradientVersion import *
from processing_helpers import *
from analysis_helpers import *
from triplets import *
from mri_maps import *
from enrichments import *
from brainspan import *
%R source("../code/Rplots.R");
%R source("../code/brainPlots.R");
%R source("../code/enrichmentsPlots.R");
%R source("../code/plot_maps.R");
%R source("../code/brainspanPlots.R");

In [6]:
dm_hcp_3d_ds5 = gradientVersion(approach='dm', kernel='normalized_angle', alpha=1).fit('hcp_3donors_ds5')
dm_hcp_ds9 = gradientVersion(approach='dm', kernel='normalized_angle', alpha=1).fit('hcp_ds9')

New gradients version: method=dm, kernel=normalized_angle, sparsity=0, data=hcp_3donors_ds5
New gradients version: method=dm, kernel=normalized_angle, sparsity=0, data=hcp_ds9


## Use Justine's compiled version?

Wrong parcellation ... sigh

In [52]:
from pymatreader import read_mat
from scipy.io import loadmat

In [65]:
terms = [x[0] for x in loadmat('../data/neurosynth/terms.mat')['terms'][0][0][2][0]]

In [68]:
loadmat('../data/neurosynth/neurosynth.mat')['cogact125']

(234, 123)

## Try to use neurosynth directly...

Try to parcellate neurosynth

In [98]:
dict(np.load('../data/neurosynth/data-neurosynth_version-7_vocab-terms_source-abstract_type-tfidf_features.npz'))

{'indices': array([   23,   249,   338, ..., 13509, 13991, 14053], dtype=int32),
 'indptr': array([      0,     147,     288, ..., 1049122, 1049225, 1049299],
       dtype=int32),
 'format': array(b'csc', dtype='|S3'),
 'shape': array([14371,  3228]),
 'data': array([0.05539422, 0.09387571, 0.06899317, ..., 0.38408549, 0.09388443,
        0.10071574])}

In [103]:
pd.read_csv('../data/neurosynth/data-neurosynth_version-7_vocab-terms_vocabulary.txt', header=None)

,0
0,001
1,01
2,05
3,10
4,100
...,...
3223,young older
3224,younger
3225,younger adults
3226,youth


In [92]:
pd.read_csv('../data/neurosynth/data-neurosynth_version-7_coordinates.tsv', sep='\t')

,id,table_id,table_num,peak_id,x,y,z
0,9065511,28698,1.,548691,38.0,-48.0,49.0
1,9065511,28698,1.,548699,52.0,-62.0,14.0
2,9065511,28698,1.,548698,-25.0,-45.0,-2.0
3,9065511,28698,1.,548697,25.0,-35.0,-8.0
4,9065511,28698,1.,548696,42.0,-47.0,-19.0
...,...,...,...,...,...,...,...
507886,29845006,35711,05,686520,27.0,50.0,23.0
507887,29845006,35711,05,686519,-28.0,51.0,15.0
507888,29845006,35711,05,686535,10.0,-69.0,39.0
507889,29845006,35711,05,686526,36.0,16.0,4.0


In [87]:
import gzip
import pickle

with gzip.open('../data/neurosynth/neurosynth_dataset.pkl.gz', 'rb') as f:
    ns = pickle.load(f)

In [77]:
ns

Dataset(14371 experiments, space='mni152_2mm')

Put gradients into vertices

In [71]:
from nilearn.input_data import NiftiLabelsMasker

In [20]:
hcp_img_mni = nib.load("../data/parcellations/HCPMMP1_on_MNI152_ICBM2009a_nlin.nii.gz")

In [9]:
g1 = dm_hcp_ds9.scores[0].reindex(range(1,181)).values

In [ ]:
from neuromaps.nulls.spins import parcels_to_vertices
parcels_to_vertices(img, g1)

In [21]:
data = np.atleast_2d(g1)
img = NiftiLabelsMasker(hcp_img_mni).fit().inverse_transform(data)

In [30]:
nib.save(img, "../outputs/test.nii.gz")

Use nimare to get neurosynth files

In [35]:
import os
from pprint import pprint

from nimare.extract import download_abstracts, fetch_neuroquery, fetch_neurosynth
from nimare.io import convert_neurosynth_to_dataset

In [36]:
out_dir = os.path.abspath("../data")
os.makedirs(out_dir, exist_ok=True)

files = fetch_neurosynth(
    data_dir=out_dir,
    version="7",
    overwrite=False,
    source="abstract",
    vocab="terms",
)
# Note that the files are saved to a new folder within "out_dir" named "neurosynth".
pprint(files)
neurosynth_db = files[0]

[{'coordinates': '/home/rajd2/AHBA/data/neurosynth/data-neurosynth_version-7_coordinates.tsv.gz',
  'features': [{'features': '/home/rajd2/AHBA/data/neurosynth/data-neurosynth_version-7_vocab-terms_source-abstract_type-tfidf_features.npz',
                'vocabulary': '/home/rajd2/AHBA/data/neurosynth/data-neurosynth_version-7_vocab-terms_vocabulary.txt'}],
  'metadata': '/home/rajd2/AHBA/data/neurosynth/data-neurosynth_version-7_metadata.tsv.gz'}]


In [45]:
neurosynth_dset = convert_neurosynth_to_dataset(
    coordinates_file=neurosynth_db["coordinates"],
    metadata_file=neurosynth_db["metadata"],
    annotations_files=neurosynth_db["features"],
)
neurosynth_dset.save(os.path.join(out_dir, "neurosynth_dataset.pkl.gz"))
print(neurosynth_dset)

Dataset(14371 experiments, space='mni152_2mm')


In [47]:
from neurosynth import Dataset, meta, decode, network

In [48]:
neurosynth_dset.get_feature_names()

AttributeError: 'Dataset' object has no attribute 'get_feature_names'